In [ ]:
import open3d as o3d
import open3d.visualization.gui as gui
import open3d.visualization.rendering as rendering
import threading
import numpy as np
# from open3d.web_visualizer import _global_jupyter_loop

In [ ]:
class _AsyncEventLoop:

    class _Task:
        _g_next_id = 0

        def __init__(self, f):
            self.task_id = self._g_next_id
            self.func = f
            self._g_next_id += 1

    # Do not call this directly, use instance instead
    def __init__(self):
        # TODO: find a better solution. Currently py::print requires GIL which
        # causes deadlock when AsyncEventLoop is used. By calling
        # reset_print_function(), all C++ prints will be directed to the
        # terminal while python print will still remain in the cell.
        o3d.utility.reset_print_function()
        self._lock = threading.Lock()
        self._run_queue = []
        self._return_vals = {}
        self._started = False

    def start(self):
        if not self._started:
            self._thread = threading.Thread(target=self._thread_main)
            self._thread.start()
            self._started = True

    def run_sync(self, f):
        with self._lock:
            task = self._Task(f)
            self._run_queue.append(task)

        while True:
            with self._lock:
                if task.task_id in self._return_vals:
                    return self._return_vals[task.task_id]

    def _thread_main(self):
        app = gui.Application.instance
        app.initialize()

        done = False
        while not done:
            with self._lock:
                for task in self._run_queue:
                    retval = task.func()
                    self._return_vals[task.task_id] = retval
                self._run_queue.clear()

            done = not app.run_one_tick()


# The _AsyncEventLoop class shall only be used to create a singleton instance.
# There are different ways to achieve this, here we use the module as a holder
# for singleton variables, see: https://stackoverflow.com/a/31887/1255535.
_global_jupyter_loop = _AsyncEventLoop()
_global_jupyter_loop.start()

In [ ]:
def web_draw(geometry):
    
    def add_window():
        app = gui.Application.instance
        window = o3d.visualization.O3DVisualizer("Open3D", 640, 480)
        window.add_geometry("Torus", geometry)
        window.reset_camera_to_default()
        app.add_window(window)
        return window.uid
        
    uid = _global_jupyter_loop.run_sync(add_window)
    print(f"Newly add Window: {uid}")
    visualizer = o3d.WebVisualizer(window_uid=uid)
    visualizer.show()

In [ ]:
cube_red = o3d.geometry.TriangleMesh.create_box(1, 2, 4)
cube_red.compute_vertex_normals()
cube_red.paint_uniform_color((1.0, 0.0, 0.0))
web_draw(cube_red)

In [ ]:
cube_blue = o3d.geometry.TriangleMesh.create_box(1, 2, 4)
cube_blue.compute_vertex_normals()
cube_blue.paint_uniform_color((0.0, 0.0, 1.0))
web_draw(cube_blue)

In [ ]:
def torus():
    app = gui.Application.instance

    torus = o3d.geometry.TriangleMesh.create_torus()
    torus.compute_vertex_normals()
    mat = rendering.Material()
    mat.shader = "defaultLit"

    w = o3d.visualization.O3DVisualizer("Open3D", 640, 480)
    w.add_geometry("Torus", torus, mat)
    w.reset_camera_to_default()
    app.add_window(w)


def box():
    app = gui.Application.instance

    torus = o3d.geometry.TriangleMesh.create_box()
    torus.compute_vertex_normals()
    mat = rendering.Material()
    mat.shader = "defaultLit"

    w = o3d.visualization.O3DVisualizer("Open3D", 640, 480)
    w.add_geometry("box", torus, mat)
    w.reset_camera_to_default()
    app.add_window(w)

In [ ]:
wid_torus = _global_jupyter_loop.run_sync(torus)
wid_box = _global_jupyter_loop.run_sync(box)
print(f"New windows: {wid_torus}, {wid_box}")

In [ ]:
# visualizer = o3d.WebVisualizer(window_uid="window_0")
# visualizer.show()
# visualizer = o3d.WebVisualizer(window_uid="window_1")
# visualizer.show()